In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import statistics

In [779]:
class MPC(object):
    def __init__(self):
        self.g = 9.81
        self.weight_left = 30
        self.weight_right = 35
        self.angle = 0
        self.max_angle = 45
        self.min_angle = -45
        self.seesaw_left_length = 2
        self.seesaw_right_length = 2
        self.friction = 0.05

        self.time_step = 0.01  # saniye cinsinden mesela
        self.reel_vel = 0      # Başlangıç hızı
        self.min_weigth = 0
        self.max_weight = 60

        self.output_round = 2

        self.target_angle = 0

        self.time = 0
        self.times = []
        self.angles = []
        self.weight_rights = []
        self.errors = []

        self.max_output = 1
        self.min_output = -1

        self.integral = 0
        self.previous_error = 0

        self.reset_time = 9.99

        self.calc_vels= []
        self.calc_accs = []
        self.error_radians = []
        self.pred_left_weights = []

        self.pred_time = -1
        self.pred_left_weight = -1

        self.pred_time = 0
        self.pred_left_weight = 0
        self.reward = 0
        
        self.reset()

    def reset(self):
        self.angle = 0
        self.time = 0
        self.times = []
        self.weight_rights = []
        self.angles = []
        self.weight_right = 35
        self.reel_vel = 0
        self.errors = []

        self.integral = 0
        self.previous_error = 0
        
        self.calc_vels= []
        self.calc_accs = []
        self.error_radians = []
        self.pred_left_weights = []

        self.pred_time = 0
        self.pred_left_weight = 0
        self.reward = 0
        

    def pid(self, error, Kp, Ki, Kd):
        # Proportional terim
        P_out = Kp * error

        # Integral terim
        self.integral += error * self.time_step
        I_out = Ki * self.integral
    
        # Derivative terim
        derivative = (error - self.previous_error) / self.time_step
        D_out = Kd * derivative

        # Toplam çıkış
        output = round(P_out + I_out + D_out, self.output_round)

        # Önceki hatayı güncelle
        self.previous_error = error
        
        # Sağ ağırlığı güncelle
        if self.pred_time != self.time or abs(self.angle) > 0.01:
            self.reward = 0
            
            if self.weight_rights[-1] - (self.weight_rights[0] + output) > self.max_output:
                self.weight_right = self.weight_rights[-1] - self.max_output
            elif self.weight_rights[-1] - (self.weight_rights[0] + output) < self.min_output:
                self.weight_right = self.weight_rights[-1] - self.min_output
            else:
                self.weight_right = self.weight_rights[0] + output
        else:
            self.reward = 1

            self.weight_right = statistics.median([w for w in self.pred_left_weights[-5:] if w is not None]) if any(w is not None for w in self.pred_left_weights[-5:]) else None
            print(self.weight_right)
            self.weight_rights[0] = self.weight_right

        if self.weight_right < self.min_weigth:
            self.weight_right = self.min_weigth
        elif self.weight_right > self.max_weight:
            self.weight_right = self.max_weight

        return self.weight_right


    def inv_interpolate(input_values, output_values, target_y, n=4):
        # input_values ve output_values'dan None olmayanları al
        clean_data = [(x, y) for x, y in zip(input_values, output_values) if x is not None and y is not None]

        if len(clean_data) < 2:
            return None  # Yeterli veri yoksa None döndür

        # Son n veri noktasını al
        last_n_data = clean_data[-n:]

        result = 0

        for i in range(len(last_n_data)):
            xi, yi = last_n_data[i]
            term = 1

            for j in range(len(last_n_data)):
                if j != i:
                    xj, yj = last_n_data[j]
                    denominator = yi - yj
                    if denominator == 0:
                        continue  # Aynı y değeri varsa, bölme hatası engellenir
                    term *= (target_y - yj) / denominator

            result += xi * term

        return result




    def calc_velocity(self):
        if len(self.error_radians) >= 2:
            calc_vel = (self.error_radians[-1] - self.error_radians[-2]) / self.time_step
            self.calc_vels.append(calc_vel)
            return calc_vel
        else:
            self.calc_vels.append(None)
            return None
    
    def calc_acceleration(self):
        if len(self.calc_vels) >= 2 and self.calc_vels[-2] is not None and self.calc_vels[-1] is not None:
            calc_acc = (self.calc_vels[-1] - self.calc_vels[-2]) / self.time_step
            self.calc_accs.append(calc_acc)
            return calc_acc
        else:
            self.calc_accs.append(None)
            return None


    def simulate_seesaw(self, Kp, Ki, Kd):
        
        self.time += self.time_step
        
        reel_acc = ((self.seesaw_left_length * self.g * self.weight_left) - 
                    (self.seesaw_right_length * self.g * self.weight_right)) / (self.weight_left + self.weight_right)

        reel_acc = reel_acc - (self.friction * reel_acc)

        self.reel_vel += reel_acc * self.time_step
        self.angle += self.reel_vel * self.time_step

        if self.angle > np.radians(self.max_angle):
            self.angle = np.radians(self.max_angle)
            self.reel_vel = 0
        elif self.angle < np.radians(self.min_angle):
            self.angle = np.radians(self.min_angle)
            self.reel_vel = 0

        print(f"Adım: {(self.time):.2f} | Açı: {np.degrees(self.angle):.2f}° | Sağ Ağırlık: {self.weight_right:.2f}kg")


        # Verileri kaydet
        self.times.append(self.time)
        self.angles.append(self.angle)
        self.weight_rights.append(self.weight_right)

        error = np.degrees(self.angle) - self.target_angle
        error_radian = self.angle - self.target_angle
        
        self.errors.append(error)
        self.error_radians.append(error_radian)

        velocity = MPC.calc_velocity(self)
        acceleration = MPC.calc_acceleration(self)
        
        # Zamana göre interpolasyon ile tahmin zamanı hesapla
        pred_time = MPC.inv_interpolate(self.times, self.calc_vels, self.target_angle)

        # Eğer interpolasyon sonucu None dönerse, pred_time'ı None yapıyoruz
        if pred_time is not None:
            self.pred_time = round(pred_time, 2)
        else:
            self.pred_time = None
        
        self.pred_left_weight = MPC.inv_interpolate(self.weight_rights,self.calc_accs,self.target_angle)
        self.pred_left_weights.append(self.pred_left_weight)


        self.weight_right = self.pid(error, Kp, Ki, Kd)

        
        if self.time > self.reset_time:
            MPC.reset(self)

        return self.reward

In [773]:
# Nesne oluşturup çalıştırma
mpc = MPC()

In [774]:
mpc.simulate_seesaw(Kp=100, Ki=0, Kd=0)

Adım: 0.01 | Açı: -0.01° | Sağ Ağırlık: 35.00kg


0

In [780]:
# Nesne oluşturup çalıştırma
mpc = MPC()

In [786]:
for i in range(1000):
    mpc.simulate_seesaw(50,2.5,0.5)

Adım: 0.01 | Açı: -0.01° | Sağ Ağırlık: 35.00kg
Adım: 0.02 | Açı: -0.02° | Sağ Ağırlık: 34.18kg
Adım: 0.03 | Açı: -0.04° | Sağ Ağırlık: 33.18kg
Adım: 0.04 | Açı: -0.07° | Sağ Ağırlık: 32.18kg
Adım: 0.05 | Açı: -0.09° | Sağ Ağırlık: 31.18kg
Adım: 0.06 | Açı: -0.12° | Sağ Ağırlık: 30.18kg
Adım: 0.07 | Açı: -0.15° | Sağ Ağırlık: 29.18kg
Adım: 0.08 | Açı: -0.17° | Sağ Ağırlık: 28.18kg
Adım: 0.09 | Açı: -0.18° | Sağ Ağırlık: 27.18kg
Adım: 0.10 | Açı: -0.19° | Sağ Ağırlık: 26.18kg
Adım: 0.11 | Açı: -0.19° | Sağ Ağırlık: 25.18kg
Adım: 0.12 | Açı: -0.18° | Sağ Ağırlık: 25.26kg
Adım: 0.13 | Açı: -0.17° | Sağ Ağırlık: 26.17kg
Adım: 0.14 | Açı: -0.15° | Sağ Ağırlık: 27.17kg
Adım: 0.15 | Açı: -0.12° | Sağ Ağırlık: 28.17kg
Adım: 0.16 | Açı: -0.10° | Sağ Ağırlık: 29.17kg
Adım: 0.17 | Açı: -0.07° | Sağ Ağırlık: 30.17kg
Adım: 0.18 | Açı: -0.04° | Sağ Ağırlık: 31.17kg
Adım: 0.19 | Açı: -0.02° | Sağ Ağırlık: 32.17kg
Adım: 0.20 | Açı: -0.01° | Sağ Ağırlık: 33.17kg
Adım: 0.21 | Açı: -0.00° | Sağ Ağırlık: 